# Word embedding and one-hot encoding







## One-hot encoding

> One-hot encoding is the process of turning categorical factors into a numerical structure that machine learning algorithms can readily process. It functions by representing each category in a feature as a binary vector of 1s and 0s, with the vector's size equivalent to the number of potential categories.

In [30]:
data = ['cold', 'cold', 'warm', 'cold', 'hot', 'hot', 'warm', 'cold', 'warm', 'hot']

### One-hot integer encoding

In [31]:
import numpy as np
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(np.array(data))

print(data)
print(integer_encoded)

['cold', 'cold', 'warm', 'cold', 'hot', 'hot', 'warm', 'cold', 'warm', 'hot']
[0 0 2 0 1 1 2 0 2 1]


### One-hot binary encoding

In [32]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder

one_hot_encoder = OneHotEncoder(sparse_output=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded_data = one_hot_encoder.fit_transform(integer_encoded)

print(data)
print(onehot_encoded_data)

['cold', 'cold', 'warm', 'cold', 'hot', 'hot', 'warm', 'cold', 'warm', 'hot']
[[1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]


## Problem 1
What are the limitations of one-hot encoding?

- Kích thước lớn, làm tăng bộ nhớ và thời gian xử lí:
- - Nếu một cột có 10.000 giá trị khác nhau, One-Hot Encoding sẽ tạo ra 10.000 cột mới, gây lãng phí tài nguyên.
- Ma trận thưa thớt:
- - Hầu hết các giá trị trong vector One-Hot là 0, dẫn đến ma trận thưa.
- Không thể biểu diễn mối quan hệ ngữ nghĩa giữa các từ:
- - One-Hot Encoding coi tất cả các giá trị là độc lập và không có mối quan hệ nào giữa chúng.
- - Từ "vui" và "hạnh phúc" có ý nghĩa gần nhau, nhưng vector của chúng lại hoàn toàn khác nhau.
- Không thể xử lí tính phong phú của ngữ nghĩa:
- - Một từ có thể có nhiều nghĩa khác nhau tùy theo ngữ cảnh, nhưng One-Hot Encoding không thể phân biệt được.
- - One-Hot Encoding không cung cấp thông tin về ngữ cảnh của từ.
- Không thể biểu diễn thông tin thứ tự của từ trong câu.

In [33]:
# Example
words = ['vui', 'hạnh phúc', 'buồn', 'tốt', 'xấu', 'vui', 'buồn', 'hạnh phúc']
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import numpy as np

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(np.array(words))
print(words)
print(integer_encoded)

one_hot_encoder = OneHotEncoder(sparse_output=False)
integer_encoded = integer_encoded.reshape(len(words), 1)
onehot_encoded_data = one_hot_encoder.fit_transform(integer_encoded)
print(words)
print(onehot_encoded_data)

['vui', 'hạnh phúc', 'buồn', 'tốt', 'xấu', 'vui', 'buồn', 'hạnh phúc']
[3 1 0 2 4 3 0 1]
['vui', 'hạnh phúc', 'buồn', 'tốt', 'xấu', 'vui', 'buồn', 'hạnh phúc']
[[0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]]


## Word embedding

ELI5 for word embeddings
> The word embeddings can be thought of as a child’s understanding of the words. Initially, the word embeddings are randomly initialized and they don’t make any sense, just like the baby has no understanding of different words. It’s only after the model has started getting trained, the word vectors/embeddings start to capture the meaning of the words, just like the baby hears and learns different words."

In [34]:
import torch
from torch import nn
from torch.nn import functional as F

In [35]:
import pandas as pd

corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
]

### Unigram transformation

In [36]:
from nltk import ngrams
from typing import List

def ngrams_transform(document: List[str],
                     n_gram: int) -> List[str]:
    """
    N-grams transformations for a given text

    Args:
    document (List[str]) -- The document to-be-processed
    n_gram   (int)       -- Number of grams

    Returns:
    A list of string after n-grams processed
    """

    ### START YOUR CODE HERE ###
    
    tokenized_corpus = []
    for sentence in corpus:
        tokenized_sentence = []
        for word in sentence.split():
            tokenized_sentence.append(word.lower())
        tokenized_corpus.append(tokenized_sentence)

    ngram_list = []
    for sentence in tokenized_corpus:
        for grams in ngrams(sentence, n_gram):
            ngram_list.append(' '.join(grams))

    return ngram_list

    ### END YOUR CODE HERE ###

In [37]:
n_grams_list = ngrams_transform(corpus,
                                n_gram=1)
n_grams_list

['this',
 'is',
 'the',
 'first',
 'document.',
 'this',
 'document',
 'is',
 'the',
 'second',
 'document.',
 'and',
 'this',
 'is',
 'the',
 'third',
 'one.',
 'is',
 'this',
 'the',
 'first',
 'document?']

In [38]:
# Integer label for the given corpus
label_encoder = LabelEncoder()
corpus_vector = label_encoder.fit_transform(np.array(n_grams_list))

# Tensorize the input vector
example_text_tensor = torch.Tensor(corpus_vector).to(dtype=torch.long)
print(f"Example text tensor: {example_text_tensor}")
print(f"Shape of example text tensor: {example_text_tensor.shape}")

Example text tensor: tensor([10,  5,  8,  4,  2, 10,  1,  5,  8,  7,  2,  0, 10,  5,  8,  9,  6,  5,
        10,  8,  4,  3])
Shape of example text tensor: torch.Size([22])


### Create an example for embedding function to map from a word dimension to a lower dimensional space

In [39]:
num_vocab = 22 # number of vocabulary
num_dimension = 50 # dimensional embeddings

# Declare the mapping function
example_embedding_function = nn.Embedding(num_vocab, num_dimension)

In [40]:
example_output_tensor = example_embedding_function(example_text_tensor)
print(f"Embedding shape: {example_output_tensor.shape}")

Embedding shape: torch.Size([22, 50])


# Word2vec


* Word2vec is a **class of models** that represents a word in a large text corpus as a vector in n-dimensional space(or n-dimensional feature space) bringing similar words closer to each other.



* Word2vec is a simple yet popular model to construct representating embedding for words from a representation space to a much lower dimensional space (compared to the respective number of words in a dictionary).



* Word2Vec has two neural network-based variants, which are:

    * Continuous Bag of Words (CBOW)
    * Skip-gram.
![](https://kavita-ganesan.com/wp-content/uploads/skipgram-vs-cbow-continuous-bag-of-words-word2vec-word-representation-2048x1075.png)


## Continuous Bag of words (CBOW)

* The Continuous Bag-of-Words model (CBOW) is frequently used in NLP deep learning. It is a model that tries to predict words given the context of a few words before and a few words after the target word. This is distinct from language modeling, since CBOW is not sequential and does not have to be probabilistic. Typically, CBOW is used to quickly train word embeddings, and these embeddings are used to initialize the embeddings of some more complicated model. Usually, this is referred to as pretraining embeddings. It almost always helps performance a couple of percent.

* CBOW is modelled as follows:
    * Given a target word $w_i$ and an $N$ context window on each side, $w_{i-1}, \cdots, w_{i-N}$ and $w_{i+1},\cdots, w_{i+N}$, referring to all context words collectively as $C$.

    * CBOW tries to minimize the objective function:

$$
-\log p(w_i|C) = -\log\text{Softmax}\left(A\left(\sum_{w\in C}q_w\right)+b\right)
$$

where $q_w$ is the embedding of word $w$.

In [41]:
# N = 2 according to the definition
CONTEXT_SIZE = 2

corpus = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells."""

corpus = corpus.split()
len(corpus)

62

### Create an integer mapping

In [42]:
vocab = set(corpus)
vocab_size = len(vocab)

# Integer word mapping
word_to_idx = {word: i for i, word in enumerate(vocab)}
word_to_idx

{'rules': 0,
 'In': 1,
 'we': 2,
 'they': 3,
 'idea': 4,
 'data.': 5,
 'computers.': 6,
 'computational': 7,
 'are': 8,
 'spirits': 9,
 'The': 10,
 'computer': 11,
 'that': 12,
 'to': 13,
 'spells.': 14,
 'process': 15,
 'People': 16,
 'direct': 17,
 'Computational': 18,
 'process.': 19,
 'inhabit': 20,
 'of': 21,
 'manipulate': 22,
 'effect,': 23,
 'other': 24,
 'study': 25,
 'beings': 26,
 'programs': 27,
 'about': 28,
 'pattern': 29,
 'program.': 30,
 'processes': 31,
 'the': 32,
 'by': 33,
 'evolve,': 34,
 'with': 35,
 'We': 36,
 'a': 37,
 'is': 38,
 'conjure': 39,
 'our': 40,
 'things': 41,
 'evolution': 42,
 'directed': 43,
 'processes.': 44,
 'abstract': 45,
 'As': 46,
 'called': 47,
 'create': 48}

### Build context according to the given corpus

In [43]:
data = []

for i in range(CONTEXT_SIZE, len(corpus) - CONTEXT_SIZE):
    context = (
        [corpus[i - j - 1] for j in range(CONTEXT_SIZE)]
        + [corpus[i + j + 1] for j in range(CONTEXT_SIZE)]
    )
    target = corpus[i]
    data.append((context, target))

data

[(['are', 'We', 'to', 'study'], 'about'),
 (['about', 'are', 'study', 'the'], 'to'),
 (['to', 'about', 'the', 'idea'], 'study'),
 (['study', 'to', 'idea', 'of'], 'the'),
 (['the', 'study', 'of', 'a'], 'idea'),
 (['idea', 'the', 'a', 'computational'], 'of'),
 (['of', 'idea', 'computational', 'process.'], 'a'),
 (['a', 'of', 'process.', 'Computational'], 'computational'),
 (['computational', 'a', 'Computational', 'processes'], 'process.'),
 (['process.', 'computational', 'processes', 'are'], 'Computational'),
 (['Computational', 'process.', 'are', 'abstract'], 'processes'),
 (['processes', 'Computational', 'abstract', 'beings'], 'are'),
 (['are', 'processes', 'beings', 'that'], 'abstract'),
 (['abstract', 'are', 'that', 'inhabit'], 'beings'),
 (['beings', 'abstract', 'inhabit', 'computers.'], 'that'),
 (['that', 'beings', 'computers.', 'As'], 'inhabit'),
 (['inhabit', 'that', 'As', 'they'], 'computers.'),
 (['computers.', 'inhabit', 'they', 'evolve,'], 'As'),
 (['As', 'computers.', 'evol

### Problem 2
Name at least 2 limitations at this context construction step? Explain your answers.

1. Mất trật tự của ngữ cảnh:
- Khi ghép hai danh sách (trước và sau từ trung tâm), thứ tự thực tế của các từ trong câu có thể bị đảo lộn, khiến mô hình khó học được cú pháp chính xác. Làm mất đi ý nghĩa ngữ pháp của câu.
2. Các từ nằm vị trí xa bị bỏ qua:
- Phương pháp trên chỉ lấy 1 số từ cố định trước hoặc sau từ target, điều này khiến cho các từ nằm xa có thể bị bỏ qua dù mang ý nghĩa quan trọng.
- Ví dụ: "Bạn Tuấn, dù nhà ở xa nhưng vẫn luôn đi học đúng giờ.". Cụm từ "luôn đi học đúng giờ" trong câu trên mới mang ý chính của câu. Tuy nhiên, nếu ta chọn target word là "Tuấn" thì CBOW chỉ dự đoán được "Bạn", "dù", "nhà" và không nắm bắt được ý chính của câu.
3. Không xử lý được từ đa nghĩa:
- Ví dụ: "Tôi không thích uống nước đá." và "Tôi thích đá bóng.". Từ "đá" trong 2 câu trên mang nghĩa hoàn toàn khác nhau nhưng CBOW chỉ học duy nhất vector "đá" nên không thể phân biệt 2 nghĩa này.
4. Lặp lại target nhiều lần khiến cho mô hình dễ bị overfit gây ra việc khó phân biệt ngữ nghĩa thực tế.

### Vectorize context

In [44]:
def make_context_vector(context: List[str],
                        word_to_idx: dict) -> torch.Tensor:
    """
    Function to map a word context vector into a torch tensor

    Args:
    context (List[str]) -- A context (including individual n-grams tokens)
    word_to_idx (dict)  -- A functionto map a word into its respective integer

    Returns:
    A pytorch tensor including a list of mapped word

    Example:
    ['are', 'We', 'to', 'study'] --> tensor([40, 22, 27, 47])
    """

    ### START YOUR CODE HERE ###

    list_idx = []
    for word in context:
        if word in word_to_idx:
            list_idx.append(word_to_idx[word])

    return torch.tensor(list_idx, dtype=torch.long)

    ### END YOUR CODE HERE ###

In [45]:
# Functional test
print("Example sample: ", data[0][0])
make_context_vector(data[0][0], word_to_idx)

Example sample:  ['are', 'We', 'to', 'study']


tensor([ 8, 36, 13, 25])

### CBOW model implementation

In [46]:
class CBOW(nn.Module):
    def __init__(self,
                 vocab_size: int,
                 embed_dim: int) -> None:
        """
        Model constructor
        """
        super().__init__()

        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

        self.embedding_layer = nn.Embedding(vocab_size, embed_dim)
        self.linear_layer = nn.Linear(embed_dim, vocab_size)

        # Neural weight initialization
        nn.init.xavier_normal_(self.embedding_layer.weight)
        nn.init.xavier_normal_(self.linear_layer.weight)

    def forward(self, inputs):
        """
        Function to conduct forward passing
        """
        embedding = self.embedding_layer(inputs)
        embedding = torch.sum(embedding, dim=1)
        output = self.linear_layer(embedding)
        output_softmax = F.log_softmax(output, dim=1)
        return output_softmax

In [47]:
cbow_model = CBOW(vocab_size=vocab_size,
                  embed_dim=10)

# Enable gradient for model training
cbow_model.train()
cbow_model

CBOW(
  (embedding_layer): Embedding(49, 10)
  (linear_layer): Linear(in_features=10, out_features=49, bias=True)
)

### Train

#### Hyperparameters and training configuration

In [48]:
num_epochs: int = 5
learning_rate: float = 5e-2
optimizer: torch.optim = torch.optim.Adam(cbow_model.parameters(),
                                          lr=learning_rate)

loss_function = nn.NLLLoss()

#### Training phase

In [49]:
for epoch in range(1, num_epochs + 1):
    print(f"#Epoch {epoch}/{num_epochs}")

    # Construct input and target tensor
    input_vector, target_vector = torch.tensor(make_context_vector(data[0][0], word_to_idx)), torch.tensor(word_to_idx[data[0][1]])
    input_vector = input_vector.unsqueeze(0)
    target_vector = target_vector.unsqueeze(0)

    # Join whole data into 1 tensor set
    for idx in range(1, len(data)):
        input_tensor = torch.tensor(make_context_vector(data[idx][0], word_to_idx)).unsqueeze(0)
        target_tensor = torch.tensor(word_to_idx[data[idx][1]]).unsqueeze(0)
        torch.cat((input_vector, input_tensor), 0)
        torch.cat((target_vector, target_tensor), 0)

    # Zero out the gradients from the old instance to avoid tensor accumulation
    cbow_model.zero_grad()

    # Forward passing
    log_probabilities = cbow_model(input_vector)

    # Evaluate loss
    loss = loss_function(log_probabilities, target_vector)

    # Backpropagation
    loss.backward()

    # Update the gradient according to the optimization algorithm
    optimizer.step()

    # Get loss values
    epoch_loss = loss.item()
    print("Loss:", epoch_loss)

#Epoch 1/5
Loss: 3.884511709213257
#Epoch 2/5
Loss: 3.112879753112793
#Epoch 3/5
Loss: 2.1810035705566406
#Epoch 4/5
Loss: 1.1467602252960205
#Epoch 5/5
Loss: 0.34934601187705994


C:\Users\Admin\AppData\Local\Temp\ipykernel_4052\2192199030.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_vector, target_vector = torch.tensor(make_context_vector(data[0][0], word_to_idx)), torch.tensor(word_to_idx[data[0][1]])
C:\Users\Admin\AppData\Local\Temp\ipykernel_4052\2192199030.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_tensor = torch.tensor(make_context_vector(data[idx][0], word_to_idx)).unsqueeze(0)


#### Inference

In [50]:
with torch.no_grad(): # No gradient update in inference
    context = ['In', 'processes.', 'we', 'conjure']

    # Vectorize input from text to numeric type
    input_tensor = torch.tensor(make_context_vector(context, word_to_idx)).unsqueeze(0)

    # Model makes prediction
    output_tensor = cbow_model(input_tensor)

    # Get the item id with the highest probability
    prediction = torch.argmax(output_tensor).detach().tolist()

    # Query the respective word from the given item id
    key_list = list(word_to_idx.keys())
    prediction = key_list[prediction]

    print("Context:", context)
    print("Prediction:", prediction)

Context: ['In', 'processes.', 'we', 'conjure']
Prediction: about


C:\Users\Admin\AppData\Local\Temp\ipykernel_4052\1681364246.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_tensor = torch.tensor(make_context_vector(context, word_to_idx)).unsqueeze(0)


## Skip-gram

<center>
<img src="https://machinelearningcoban.com/tabml_book/_images/word2vec2.png">
</center>

- Skip gram is based on the distributional hypothesis where words with similar distribution is considered to have similar meanings. Researchers of skip gram suggested a model with less parameters along with the novel methods to make optimization step more efficient.

- Vanilla SkipGram model:

<center>
<img src="https://d3i71xaburhd42.cloudfront.net/a1d083c872e848787cb572a73d97f2c24947a374/5-Figure1-1.png" scale=70%>
</center>

- Main idea is to optimize model so that if it is queried with a word, it should correctly guess all the context (context = 2 in the figure) words. That is,
$$
y=\sigma(Ux)
$$
    - where $x$, $y$ are one-hot encoded word vector, $U$ is the embedding matrix, and $\sigma(\cdot)$ is the softmax function.

With the same dataset, training set for skip gram can be much larger than that of NPLM since it can have $2c$ samples $\left(w_t:w_{t-c}, ...,w_t:w_{t-1},w_t:w_{t+1},...,w_{t+c}\right)$ while other n-gram based models have one $\left((w_{t-c},...w_{t-1},w_{t+1},...,w_{t+c}):w_t\right)$.

In [51]:
corpus = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells."""

In [52]:
class SkipGramModel(nn.Module):
    def __init__(self,
                 vocab_size: int,
                 embed_dim: int) -> None:
        """
        Model construction
        """
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

        ### START YOUR CODE HERE ###
        # Declare embedding function u and v
        # with given vocab size and embed dim using nn.Embedding
        self.v_embedding_layer = nn.Embedding(self.vocab_size, self.embed_dim)
        self.u_embedding_layer = nn.Embedding(self.vocab_size, self.embed_dim)

        # Network weight initialization with Xavier initialization
        nn.init.xavier_uniform_(self.v_embedding_layer.weight)
        nn.init.xavier_uniform_(self.u_embedding_layer.weight)

        ### END YOUR CODE HERE ###

    def forward(self, center_words, context):
        """
        Function to perform forward passing
        """
        v_embedding = self.v_embedding_layer(center_words)
        u_embedding = self.u_embedding_layer(context)

        score = torch.mul(v_embedding, u_embedding)
        score = torch.sum(score, dim=1)
        log_score = F.logsigmoid(score)
        return log_score

In [53]:
skipgram_model = SkipGramModel(vocab_size=vocab_size,
                               embed_dim=128)

skipgram_model.train()
skipgram_model

SkipGramModel(
  (v_embedding_layer): Embedding(49, 128)
  (u_embedding_layer): Embedding(49, 128)
)

### Prepare training data to match the format of SkipGram model

In [54]:
def gather_training_data(corpus,
                         word_to_idx: dict,
                         context_size: int):
    """
    This function is to transform the given corpus
    into the correct format for SkipGram to serve as its input
    """

    training_data = []
    all_vocab_indices = list(range(len(word_to_idx)))

    split_text = corpus.split('\n')

    # For each sentence
    for sentence in split_text:
        indices = []
        indices = [word_to_idx[word] for word in sentence.split(' ')]

        # For each word treated as center word
        for center_word_pos in range(len(indices)):

            # For each window  position
            for w in range(-context_size, context_size+1):
                context_word_pos = center_word_pos + w

                # Make sure we dont jump out of the sentence
                if context_word_pos < 0 or context_word_pos >= len(indices) or center_word_pos == context_word_pos:
                    continue

                context_word_idx = indices[context_word_pos]
                center_word_idx  = indices[center_word_pos]

                # Same words might be present in the close vicinity of each other. we want to avoid such cases
                if center_word_idx == context_word_idx:
                    continue

                training_data.append([center_word_idx, context_word_idx])

    return training_data

In [55]:
training_data = gather_training_data(corpus,
                                     word_to_idx,
                                     context_size=2)
training_data = torch.tensor(training_data).to(dtype=torch.long)
training_data.shape

torch.Size([212, 2])

### Hyperparamters and training configuration

In [56]:
num_epochs: int = 200
learning_rate: float = 5e-1
optimizer: torch.optim = torch.optim.SGD(skipgram_model.parameters(),
                                          lr=learning_rate)

### Training phase

In [57]:
for epoch in range(num_epochs + 1):
    """
    Adapt the given CBOW training code for SkipGram
    Following by the instruction comments, or you could do it on your own ;)
    """
    ### START YOUR CODE HERE ###

    # Construct input and target tensor
    input_tensor = torch.tensor(training_data[:, 0], dtype=torch.long)
    target_tensor = torch.tensor(training_data[:, 1], dtype=torch.long)

    # Zero out the gradients from the old instance to avoid tensor accumulation
    skipgram_model.zero_grad()

    # Forward passing
    logsoftmax_prediction = skipgram_model(input_tensor, target_tensor)

    # Evaluate loss (Negative log likelihood)
    loss = torch.mean(-1 * logsoftmax_prediction)

    # Backpropagation
    loss.backward()

    # Update the gradient according to the optimization algorithm
    optimizer.step()

    # Get loss values
    epoch_loss = loss.item()

    # Log result
    if epoch % 50 == 0:
        print(f"#Epoch {epoch}/{num_epochs}")
        print("Loss:", epoch_loss)

    ### END YOUR CODE HERE ###

#Epoch 0/200
Loss: 0.6899917721748352
#Epoch 50/200
Loss: 0.5998015403747559
#Epoch 100/200
Loss: 0.5147236585617065
#Epoch 150/200
Loss: 0.4317810833454132
#Epoch 200/200
Loss: 0.35444021224975586


C:\Users\Admin\AppData\Local\Temp\ipykernel_4052\167642886.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_tensor = torch.tensor(training_data[:, 0], dtype=torch.long)
C:\Users\Admin\AppData\Local\Temp\ipykernel_4052\167642886.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target_tensor = torch.tensor(training_data[:, 1], dtype=torch.long)


### Inference

In [58]:
with torch.no_grad():
    context = ['we']

    ### START YOUR CODE HERE ###
    # Based on the given inference code in the previous section, training code and the context
    # Implement the inference flow from the given context to an output word

    embedding_matrix = skipgram_model.v_embedding_layer.weight
    input_tensor = torch.tensor(word_to_idx[context[0]]).unsqueeze(0)
    input_embedding = embedding_matrix[input_tensor]
    predict_matrix = F.log_softmax(input_embedding @ embedding_matrix.T, dim=1)
    context_size = 2
    top_indices = torch.topk(predict_matrix, 2 * context_size + 1, sorted=True).indices
    key_list = list(word_to_idx.keys())
    prediction = [key_list[idx] for idx in top_indices[0]]
    prediction = prediction[1:]

    ### END YOUR CODE HERE ###
    print("Context:", context)
    print("Prediction:", prediction)

Context: ['we']
Prediction: ['are', 'study', 'of', 'spirits']


## Problem 3
What are the differences between CBOW and Skip-gram?

1. CBOW
- Dự đoán từ trung tâm (target word) dựa vào các từ ngữ cảnh (context words).
- Hiệu quả với tập dữ liệu lớn, tốc độ nhanh hơn do số lượng mẫu huấn luyện ít hơn.
- Dữ liệu nhỏ, tốc độ huấn luyện nhanh, hoạt động tốt với từ phổ biến.

2. Skip-gram
- Dự đoán các từ ngữ cảnh dựa vào từ trung tâm.
- Hiệu quả với tập dữ liệu nhỏ, có thể học tốt hơn trong trường hợp dữ liệu thưa thớt nhưng tốc độ chậm.
- Dữ liệu lớn, hiệu quả hơn với từ hiếm.